# SOLUTION OF PUZZLE'S PROBLEM

In [ ]:
from collections import namedtuple
from random import shuffle
from queue import PriorityQueue  # PriorityQueue per la coda
import numpy as np
from tqdm.auto import tqdm

In [ ]:
PUZZLE_DIM = 4
action = namedtuple('Action', ['pos1', 'pos2']) # namedtuple per le azioni
goal_state = tuple(i for i in range(1, PUZZLE_DIM**2)) + (0,) # goal state

The code solves the sliding puzzle using the A* algorithm. 

1. Initialize a random puzzle

2. verify that it is solvable

3. explore possible states using the Manhattan distance as a heuristic. 

A* expands the states with the lowest cost until the solution is found, which consists of a sequence of moves to correctly arrange the pieces of the puzzle. If the solution is found, print the necessary moves.

In [ ]:
# Trova le azioni disponibili (spostamenti possibili)
def available_actions(state: tuple) -> list:
    state_2d = np.array(state).reshape(PUZZLE_DIM, PUZZLE_DIM)
    x, y = np.where(state_2d == 0)  # Trova il vuoto
    x, y = int(x), int(y)
    actions = []
    if x > 0: actions.append(action((x, y), (x - 1, y)))  # Su
    if x < PUZZLE_DIM - 1: actions.append(action((x, y), (x + 1, y)))  # Giù
    if y > 0: actions.append(action((x, y), (x, y - 1)))  # Sinistra
    if y < PUZZLE_DIM - 1: actions.append(action((x, y), (x, y + 1)))  # Destra
    return actions

# Esegue un'azione (scambia due posizioni nel puzzle)
def do_action(state: tuple, action: 'Action') -> tuple:
    state_2d = np.array(state).reshape(PUZZLE_DIM, PUZZLE_DIM)
    x1, y1 = action.pos1
    x2, y2 = action.pos2
    state_2d[x1, y1], state_2d[x2, y2] = state_2d[x2, y2], state_2d[x1, y1]
    return tuple(state_2d.flatten())

# Calcola la distanza Manhattan (euristica)
def manhattan_distance(state: tuple) -> int:
    distance = 0
    state_2d = np.array(state).reshape(PUZZLE_DIM, PUZZLE_DIM)
    for i in range(PUZZLE_DIM):
        for j in range(PUZZLE_DIM):
            value = state_2d[i, j]
            if value != 0:  # Ignora il vuoto
                goal_x, goal_y = divmod(value - 1, PUZZLE_DIM)
                distance += abs(i - goal_x) + abs(j - goal_y)
    return distance

# Algoritmo A* per risolvere il puzzle
def a_star_search(start_state: tuple) -> list:
    pq = PriorityQueue()
    pq.put((manhattan_distance(start_state), 0, start_state, []))
    visited = set()
    visited.add(start_state)
    while not pq.empty():
        heuristic, cost, state, path = pq.get()
        if state == goal_state:
            return path, state
        for act in available_actions(state):
            new_state = do_action(state, act)
            if new_state not in visited:
                visited.add(new_state)
                new_cost = cost + 1
                pq.put((new_cost + manhattan_distance(new_state), new_cost, new_state, path + [act]))
    return [], start_state

# Controlla se lo stato è risolvibile
def is_solvable(state: tuple) -> bool:
    state = [x for x in state if x != 0]  # Ignora il vuoto
    inversions = sum(1 for i in range(len(state)) for j in range(i + 1, len(state)) if state[i] > state[j])
    return inversions % 2 == 0

# Randomizza il puzzle e garantisce che sia risolvibile
def randomize_puzzle() -> tuple:
    state = tuple(i for i in range(1, PUZZLE_DIM**2)) + (0,)
    state = list(state)
    shuffle(state)
    while not is_solvable(tuple(state)):
        shuffle(state)
    return tuple(state)

# Main
if __name__ == "__main__":
    state = randomize_puzzle()
    print("Randomized Puzzle State:")
    print(np.array(state).reshape(PUZZLE_DIM, PUZZLE_DIM))
    solution, final_state = a_star_search(state)
    if solution:
        print(f"Solution found in {len(solution)} moves!")
        for step, act in enumerate(solution, start=1):
            print(f"Step {step}: Move {act.pos1} -> {act.pos2}")
    else:
        print("No solution found.")
    print("Final State:")
    print(np.array(final_state).reshape(PUZZLE_DIM, PUZZLE_DIM))

